In [1]:
import pandas as pd
import re

def extract_info(pattern, text):
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        print(f"Failed to find information matching pattern: {pattern}")
        return None

def append_information(df, output_text, choice, human_feedback):
    # Extract information from the text using regex
    summary = extract_info('Overall Summary:(.*?)(Status Evaluation:|$)', output_text)
    status_evaluation = extract_info('Status Evaluation:(.*?)(Task Choice 1:|$)', output_text)
    task_choice_1 = extract_info('Task Choice 1:(.*?)(Task Choice 2:|$)', output_text)
    task_choice_2 = extract_info('Task Choice 2:(.*?)(Task Choice 3:|$)', output_text)
    task_choice_3 = extract_info('Task Choice 3:(.*?)(Task Choice [4-9]:|$)', output_text)
    
    # If any information could not be extracted, stop the function
    if any(info is None for info in [summary,  status_evaluation, task_choice_1, task_choice_2, task_choice_3]):
        return df

    # Construct the next prompt based on the provided template and extracted information
    last_task = ''
    if choice == 1:
        last_task = task_choice_1
    elif choice == 2:
        last_task = task_choice_2
    elif choice == 3:
        last_task = task_choice_3
    else:
        print('Invalid choice value.')
        return df

    next_prompt = f"""
You are a project manager AI named Atlas, in charge of a research team in a chemistry lab. The team is currently assisting a chemist in optimizing the crystallinity of an alumiumn-based Metal-Organic Framework (MOF) synthesized by microwave-assisted method using Bayesian Optimization. Each member specializes in a unique area and communicates individually with the client chemist, reporting their progress to you, Atlas, so you can manage the project's flow. Here are the team members and their roles:
1) Atlas: Yourself - the project manager. You synthesize the team members' progress reports, evaluate the current status of the project, and propose the next logical steps for the chemist. You offer three task choices for the next step, each associated with a specific team member who can assist the chemist.
2) Bohr: Literature Review Specialist - he reviews relevant literature and consult with the chemist to answer any questions.
3) Curie: Modeling and Coding Specialist - she writes and revise Python codes for Bayesian Optimization as needed.
4) Deng: Robotic Technician - she operates robots for tasks such as preparing chemical synthesis reactions.
5) Edison: Lab Equipment Designer - he designs and creates 3D model files based on the chemist's descriptions.
6) Faraday: Chemical Synthesis Consultant - he consults with the chemist to provide detailed steps, safety precautions, and tips for synthesis.
7) Gauss: Analytical Assistant - he analyzes and interpret data based on observations, aiding the chemist in visualizing the data obtained from the experiments.

Your recent task suggestion, the chemist's feedback and overall summary are as follows:
...
Task: {last_task}
Feedback: {human_feedback}
Overall Summary: {summary}
...

Based on the information given, you are expected to gerneate the following:
Overall Summary: Construct an updated summary that primarily draw from the previous summary, without changing too much the original summary, adding after that sentences regarding the latest report from the teammember, the latest task and the chemist's feedback. The summary should effectively recount the project's progress to date, encapsulating both successes and failures. Ensure to retain and highlight the vital details in the summary. There is no word limit for the overall summary.
Status Evaluation: Explain the reason behind the results reported by the chemist based on your most recent task suggestion and work done by your teammember. This should be a short (one or two sentence) analysis. Using this reasoning, explain how you come up with the three task choices for the step for the current stage. 
Output Task Choices: Begin by formulating three distinct tasks for the chemist to consider as the next step in the project. Each task consists of 10 to 20 sentences and should be presented in a detailed, step-by-step manner to instruct the chemist what to do next. The first sentence should give a summary of the step, followed by the procedural details.  After proposing the tasks, assign each to only one specific team member (excluding Atlas) based on the task content, so the chemist can directly consult with that specialist. Note that you should frist propose tasks based on evluation and then decide the specialiist assignment, and it's possible that more than one tasks may align best with the same specialist's area of expertise. 

Your response should follow this format:
...
Overall Summary: <updated summary>
Status Evaluation: <reasoning>
Task Choice 1: <next task choice 1> - Assigned to: <team member>
Task Choice 2: <alternative next task choice> - Assigned to: <team member>
Task Choice 3: <alternative next task choice> - Assigned to: <team member>
...

    """
    # Append the new row to the DataFrame
    new_row = pd.DataFrame({
        "Summary": [summary], 
        "Status Evaluation": [status_evaluation], 
        "Choice 1": [task_choice_1], 
        "Choice 2": [task_choice_2], 
        "Choice 3": [task_choice_3], 
        "Choice Selected": [choice],
        "Human Feedback": [human_feedback],
        "Next Prompt": [next_prompt],
    })
    
    print(next_prompt)


    # Append the new row to the DataFrame
    new_row = pd.DataFrame({
        "Summary": [summary], 
        "Status Evaluation": [status_evaluation], 
        "Choice 1": [task_choice_1], 
        "Choice 2": [task_choice_2], 
        "Choice 3": [task_choice_3], 
        "Choice Selected": [choice],
        "Human Feedback": [human_feedback],
        "Next Prompt": [next_prompt],
    })

    df = pd.concat([df, new_row], ignore_index=True)
    
    return df






In [2]:
df = pd.DataFrame(columns=["Summary", "Status Evaluation", "Choice 1", "Choice 2", "Choice 3", "Choice Selected", "Human Feedback", "Next Prompt"])
df

,Summary,Status Evaluation,Choice 1,Choice 2,Choice 3,Choice Selected,Human Feedback,Next Prompt
